In [2]:
import os, os.path
import numpy as np
import pandas as pd
#from model_attributes import *
import model_attributes as ma
from attribute_table import AttributeTable
import model_afolu as mafl
import model_ippu as mi
import model_circular_economy as mc
import model_energy as me
import model_socioeconomic as se
from model_socioeconomic import Socioeconomic
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
from typing import Union
import inspect
import ingestion as ing
import logging
from sisepuede_file_structure import *


#importlib.reload(ma)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(mafl)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(me)
importlib.reload(se)


import sisepuede as ssp
def _setup_logger(namespace: str, fn_out: Union[str, None] = None) -> None:
    global logger
    
    format_str = "%(asctime)s - %(levelname)s - %(message)s"
    # configure
    if fn_out is not None:
        logging.basicConfig(
            filename = fn_out,
            filemode = "w",
            format = format_str,
            level = logging.DEBUG
        )
    else:
        logging.basicConfig(
            format = format_str,
            level = logging.DEBUG
        )
        
    logger = logging.getLogger(namespace)
    # create console handler and set level to debug
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    # create formatter
    formatter = logging.Formatter(format_str)
    # add formatter to ch
    ch.setFormatter(formatter)
    # add ch to logger
    logger.addHandler(ch)

    return logger

_setup_logger(__name__, os.path.join(os.getcwd(), "log_temp.log"))

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:1945: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:1945: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


<Logger __main__ (DEBUG)>

In [154]:
##########################
#   LOAD INPUT TABLES    #
##########################

##  setup location of calibrated files by sector


df_fake_data = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))
dir_calibs = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo"
dict_calibration_file_paths = {
    "af": os.path.join(dir_calibs, "af", "data_complete_future_2022_09_30_test_updated.csv"),
    "ce": os.path.join(dir_calibs, "ce", "data_complete_future_2022_12_13_test.csv"),
    "en": os.path.join(dir_calibs, "en", "data_complete_future_2023_01_30_with_transformations.csv"),#"data_complete_future_2022_12_09_test.csv"),
    "ip": os.path.join(dir_calibs, "ip", "data_ippu_2023_01_16_fixed_cleaned.csv"),
    # use most recent for socioeconomic
    "se": os.path.join(dir_calibs, "en", "data_complete_future_2022_12_09_test.csv")
}


# initialize 
all_regions = None
dict_isos = {}
dict_calibration_tables = {}
dict_replace_iso = sa.model_attributes.dict_attributes.get("region").field_maps.get("region_abbreviation_to_region")
field_iso = "iso_code3"
field_region = "nation"
fields_drop = ["iso_code3", "year", "nation"] # only apply later
field_time_period = sa.model_attributes.dim_time_period

# load in tables and make some quick modifications
dict_sets = {}
for k in dict_calibration_file_paths.keys():
    # read and clean columns
    df_read = pd.read_csv(dict_calibration_file_paths.get(k))
    dict_rnm = dict((x, x.lower()) for x in df_read.columns if x != x.lower())
    df_read.rename(columns = dict_rnm, inplace = True)
    
    # filter time periods and do some field cleaning
    df_read = df_read[df_read[field_time_period] >= 0].reset_index(drop = True)
    df_read[field_region] = [dict_replace_iso.get(x.lower()) for x in list(df_read[field_iso])]
    dict_isos.update({k: set(df_read[field_iso])})
    
    # fields missing from input file: take from fake data
    fields_missing = list(set(df_fake_data.columns) - set(df_read.columns))
    fields_eliminate = list((set(df_read.columns) - set(df_fake_data.columns)) - set(fields_drop) - set(["strategy_id"]))
    
    df_read = pd.merge(
        df_read, 
        df_fake_data[[field_time_period] + fields_missing],
        on = [field_time_period],
        how = "left"
    ).drop(fields_eliminate, axis = 1)
    
    dict_calibration_tables.update({k: df_read})
    set_merge = set(df_read[field_region])
    dict_sets.update({k: set_merge})
    all_regions = set_merge if (all_regions is None) else (all_regions & set_merge)

    
attr_region = sa.model_attributes.dict_attributes.get("region")
attr_sector = sa.model_attributes.dict_attributes.get("abbreviation_sector")
attr_strat = sa.model_attributes.dict_attributes.get(f"dim_{sa.model_attributes.dim_strategy_id}")
all_regions = sorted(list(set(attr_region.key_values) & all_regions))



In [155]:
df_full = dict_calibration_tables.get("en");

strategy_verify = 3007
df_verify = df_full[
    df_full["strategy_id"].isin([strategy_verify])
].reset_index(drop = True)
var = "Transportation Mode Fuel Fraction Electricity"
sa.model_attributes.get_standard_variables(
    df_verify, 
    var
).tail()

,frac_trns_fuelmix_aviation_electricity,frac_trns_fuelmix_powered_bikes_electricity,frac_trns_fuelmix_public_electricity,frac_trns_fuelmix_rail_freight_electricity,frac_trns_fuelmix_rail_passenger_electricity,frac_trns_fuelmix_road_heavy_freight_electricity,frac_trns_fuelmix_road_heavy_regional_electricity,frac_trns_fuelmix_road_light_electricity,frac_trns_fuelmix_water_borne_electricity
787,0.0,0.03,0.000894,0.0,0.522293,0.0,0.005,0.588148,0.0
788,0.0,0.03,0.000894,0.0,0.522293,0.0,0.005,0.616111,0.0
789,0.0,0.03,0.000894,0.0,0.522293,0.0,0.005,0.644074,0.0
790,0.0,0.03,0.000894,0.0,0.522293,0.0,0.005,0.672037,0.0
791,0.0,0.03,0.000894,0.0,0.522293,0.0,0.005,0.700000,0.0


In [68]:


#####################################################################
#    INITIALIZE AN EMPTY INPUT TEMPLATE OBJECT, USE FOR BUILDING    #
#####################################################################



file_struct = SISEPUEDEFileStructure();
dir_templates = file_struct.dict_data_mode_to_template_directory.get("calibrated")
dir_templates_demo = file_struct.dict_data_mode_to_template_directory.get("demo")
bid = ing.BaseInputDatabase(dir_templates, sa.model_attributes, None, demo_q = False);
bid_demo = ing.BaseInputDatabase(dir_templates_demo, sa.model_attributes, list(all_regions)[0], demo_q = True);




In [156]:
importlib.reload(ing)
warnings.filterwarnings("ignore")
input_template = ing.InputTemplate(
    None,
    sa.model_attributes
)

########################################################
#   LOOP OVER SECTOR/REGION TO CONVERT TO TEMPLATES    #
########################################################
importlib.reload(sf)
importlib.reload(ing)
input_template = ing.InputTemplate(
    None,
    sa.model_attributes
)
dict_sector_abv_to_sector = attr_sector.field_maps.get(f"{attr_sector.key}_to_sector")

for sector_abv in ["en"]:#dict_calibration_file_paths.keys():
    
    t_0 = time.time()
    
    # get input data
    df_inputs = dict_calibration_tables.get(sector_abv)
    fields_drop_cur = [x for x in fields_drop if x in df_inputs.columns]
    sector = dict_sector_abv_to_sector.get(sector_abv)
    print(f"Starting sector {sector}...")
    
    # get baseline "demo" template, use for ranges
    fp_read = bid_demo.get_template_path(list(all_regions)[0], sector)
    df_template = pd.read_excel(
        fp_read, 
        sheet_name = input_template.name_sheet_from_index(input_template.baseline_strategy)
    )
    
    # fields to extract
    fields_ext = [x for x in input_template.list_fields_required_base]
    fields_ext += [x for x in df_template.columns if input_template.regex_template_max.match(str(x)) is not None]
    fields_ext += [x for x in df_template.columns if input_template.regex_template_min.match(str(x)) is not None]
    df_template = df_template[fields_ext].drop_duplicates()
    
    # loop over regions to build template
    for region in all_regions:
        
        # get input component and add baseline strategy marker
        fields_sort = [attr_strat.key, field_time_period] if (attr_strat.key in df_inputs.columns) else [field_time_period]
        df_input = df_inputs[
            (df_inputs[field_region] == region)
        ].drop(
            fields_drop_cur, 
            axis = 1
        ).sort_values(
            by = fields_sort
        ).reset_index(
            drop = True
        )
        
        if attr_strat.key not in df_input.columns:
            df_input[attr_strat.key] = input_template.baseline_strategy
        
        # dictionary to export to excel
        dict_write = input_template.template_from_inputs(
            df_input, 
            df_template, 
            sector_abv
        )
        
        # export 
        fp_write = bid.get_template_path(
            region, 
            sector,
            create_export_dir = True
        )
        
        sf.dict_to_excel(
            fp_write,
            dict_write   
        ) if True else None
        print(f"\tCompleted region {region}.")
    
    t_elapse = sf.get_time_elapsed(t_0)
    print(f"\nSector {sector} complete in {t_elapse} seconds.\n\n")
        
        

    

Starting sector Energy...
	Completed region argentina.
	Completed region brazil.
	Completed region chile.
	Completed region colombia.
	Completed region costa_rica.
	Completed region dominican_republic.
	Completed region ecuador.
	Completed region el_salvador.
	Completed region guatemala.
	Completed region honduras.
	Completed region jamaica.
	Completed region mexico.
	Completed region nicaragua.
	Completed region panama.
	Completed region paraguay.
	Completed region peru.
	Completed region uruguay.

Sector Energy complete in 422.75 seconds.




In [65]:
df_check = dict_calibration_tables.get("en")
#df_check.to_csv()
dfl = df_check[(df_check["time_period"] == 0) & (df_check["strategy_id"] == 0)][["nation", "deminit_trde_freight_mt_km"]].copy().reset_index(drop = True)

for i in range(len(dfl)):
    nat = dfl.iloc[i].get("nation")
    mtkm = int(np.round(dfl.iloc[i].get("deminit_trde_freight_mt_km")))
    print(f"{nat}:\t{mtkm}")

argentina:	8274
bolivia:	941299
brazil:	1896532
chile:	4086
colombia:	231485
costa_rica:	39976
dominican_republic:	44410
ecuador:	121425
guatemala:	56039
honduras:	30971
haiti:	10524
jamaica:	13654
mexico:	655696
nicaragua:	16500
panama:	32624
peru:	162794
paraguay:	42170
el_salvador:	25333
suriname:	5124
trinidad_and_tobago:	25467
uruguay:	27932
venezuela:	941299


In [91]:
importlib.reload(sf)
df_cur = ing.df_cur_out
df_base = ing.dict_is.get(0)

sf.filter_df_on_reference_df_rows(
    df_cur,
    df_base,
    ["subsector", "variable", "time_period"],#'subsector', 'variable', 'time_period'],
    ["value"],
    ["variable"]
)


ere


In [96]:
df_base.iloc[0:0]

,subsector,variable,time_period,value


In [69]:
#df_template[df_template["subsector"] == sa.model_attributes.subsec_name_scoe]
importlib.reload(ing)
warnings.filterwarnings("ignore")
input_template = ing.InputTemplate(
    None,
    sa.model_attributes
)
input_template.template_from_inputs(
    df_input, 
    df_template, 
    sector_abv
)

starting strat 0
starting strat 3001
starting strat 3002
starting strat 3003
starting strat 3004
starting strat 3005
starting strat 3006
starting strat 3007
starting strat 3008
starting strat 3009
starting strat 3010
starting strat 3011
starting strat 3012
starting strat 3013
starting strat 3014
starting strat 3015
starting strat 3016
starting strat 3017
starting strat 3018
starting strat 3019
starting strat 3020


TypeError: Can only merge Series or DataFrame objects, a <class 'NoneType'> was passed

In [66]:
df_base = sa.model_attributes.build_variable_dataframe_by_sector(
    ["Energy"],
    field_subsector = input_template.field_req_subsector,
    field_variable = input_template.field_req_variable,
    include_time_periods = True
)
set(df_base[
    
    [x.startswith("scalar") for x in list(df_base["variable"])]
]["subsector"])

{'Industrial Energy', 'Stationary Combustion and Other Energy'}

In [67]:
ing.dfi

In [51]:
with open("/Users/jsyme/Desktop/untitled_5.txt", "r+") as fl:
    lines = fl.readlines()
    lines = [x for x in lines if not ("#" in x)]
    lines = [x.strip().split(":")[0] for x in lines]
    lines = [f"{x} = self.{x} if ({x} is None) else {x}" for x in lines]
    #lines = [x.strip().split(":")[0] + ": Union[str, none] = None," for x in lines]
    
for x in sorted(lines):
    print(x)

field_req_normalize_group = self.field_req_normalize_group if (field_req_normalize_group is None) else field_req_normalize_group
field_req_subsector = self.field_req_subsector if (field_req_subsector is None) else field_req_subsector
field_req_trajgroup_no_vary_q = self.field_req_trajgroup_no_vary_q if (field_req_trajgroup_no_vary_q is None) else field_req_trajgroup_no_vary_q
field_req_uniform_scaling_q = self.field_req_uniform_scaling_q if (field_req_uniform_scaling_q is None) else field_req_uniform_scaling_q
field_req_variable = self.field_req_variable if (field_req_variable is None) else field_req_variable
field_req_variable_trajectory_group = self.field_req_variable_trajectory_group if (field_req_variable_trajectory_group is None) else field_req_variable_trajectory_group
field_req_variable_trajectory_group_trajectory_type = self.field_req_variable_trajectory_group_trajectory_type if (field_req_variable_trajectory_group_trajectory_type is None) else field_req_variable_trajectory_gro